In [1]:
import polars as pl
from great_tables import GT, md, html, nanoplot_options
from colorzero import *
from IPython.display import display, HTML

In [2]:
_file_location = './limited-memory-example/yellow_tripdata_2015-01.csv'
df = pl.read_csv(_file_location)

In [3]:
bw = (
    df
    .sort(pl.col('tpep_pickup_datetime'), descending=False)
    .group_by_dynamic(pl.col('tpep_pickup_datetime').str.to_datetime('%Y-%m-%d %H:%M:%S'), every="1w")
    .agg(
        pl.col('passenger_count').mean().alias('average_passenger_count'),
        pl.col('trip_distance').mean().alias('average_trip_distance'),
        pl.col('fare_amount').mean(),
        pl.col('extra').mean(),
        pl.col('mta_tax').mean(),
        pl.col('tip_amount').mean(),
        pl.col('tolls_amount').mean(),
        pl.col('total_amount').mean())
    ).rename(mapping={'tpep_pickup_datetime':'week'})

In [4]:
bw2 = (
    df
    .sort(pl.col('tpep_pickup_datetime'), descending=False)
    .group_by_dynamic(pl.col('tpep_pickup_datetime').str.to_datetime('%Y-%m-%d %H:%M:%S'), every="1d")
    .agg(pl.col('passenger_count').sum().alias('passenger_count_daily'))
    .rename(mapping={'tpep_pickup_datetime':'day'})
    .sort(pl.col('day'), descending=False)
    .group_by_dynamic(pl.col('day'), every="1w")
    .agg(pl.col('passenger_count_daily').implode())
    .with_columns(pl.col('passenger_count_daily').list.get(0))
    .rename(mapping={'day':'week'})
    )

In [5]:
gt1 = (
    bw.join(bw2, on='week', how='inner').style
        .tab_header('NYC Taxi dataset', subtitle=f"Weekly statistics between Dec 29, 2014 to Jan 26, 2015")
        .tab_stub(rowname_col='week')
        .tab_source_note(source_note='Kaggle, July 2017. https://www.kaggle.com/competitions/nyc-taxi-trip-duration')
        .tab_spanner(label='Averages', columns=['average_passenger_count', 'average_trip_distance'])
        .tab_spanner(label='Avg. Costs', columns=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'])
        .fmt_currency(columns=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'])
        .fmt_date(columns=['week'], date_style='m_day_year')
        .fmt_number(columns=['average_passenger_count', 'average_trip_distance'], compact=True)
        .fmt_nanoplot(columns='passenger_count_daily', autoscale=True)
        .cols_label(
            average_passenger_count=html('Passenger<br>count'),
            average_trip_distance=html('Trip<br>distance'),
            fare_amount=html('Fare'),
            extra='Extra',
            mta_tax='MTA tax',
            tip_amount='Tip',
            tolls_amount=html('Tolls'),
            total_amount=html('Total'),
            passenger_count_daily=html('Passenger count<br>(daily)')
        )
)

In [6]:
gt1

GT(_tbl_data=shape: (5, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ week      ┆ average_p ┆ average_t ┆ fare_amou ┆ … ┆ tip_amoun ┆ tolls_amo ┆ total_amo ┆ passenge │
│ ---       ┆ assenger_ ┆ rip_dista ┆ nt        ┆   ┆ t         ┆ unt       ┆ unt       ┆ r_count_ │
│ datetime[ ┆ count     ┆ nce       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ daily    │
│ μs]       ┆ ---       ┆ ---       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---      │
│           ┆ f64       ┆ f64       ┆           ┆   ┆           ┆           ┆           ┆ list[i64 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ]        │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2014-12-2 ┆ 1.766969  ┆ 19.393444 ┆ 12.259514 ┆ … ┆ 1.33035   ┆ 0.264227  ┆ 14.909873 ┆ [690407, │
│ 9         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 606716,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 565709]  │
│ 2015-01-0 ┆ 1.675578  ┆ 22.713874 ┆ 11.767176 ┆ … ┆ 1.512062  ┆ 0.239516  ┆ 14.637098 ┆ [600760, │
│ 5         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 632469,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 719033]  │
│ 2015-01-1 ┆ 1.671457  ┆ 2.75792   ┆ 11.890466 ┆ … ┆ 2.841937  ┆ 0.248048  ┆ 16.094246 ┆ [648214, │
│ 2         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 734847,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 743288]  │
│ 2015-01-1 ┆ 1.667416  ┆ 18.76329  ┆ 11.924381 ┆ … ┆ 1.612231  ┆ 0.253735  ┆ 14.886225 ┆ [575507, │
│ 9         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 661009,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 694420]  │
│ 2015-01-2 ┆ 1.665861  ┆ 4.784368  ┆ 11.855754 ┆ … ┆ 1.592241  ┆ 0.214637  ┆ 14.783954 ┆ [375606, │
│ 6         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 232108,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 898122]  │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x11092a690>, _boxhead=Boxhead([ColInfo(var='week', type=<ColInfoTypeEnum.stub: 2>, column_label='week', column_align='right', column_width=None), ColInfo(var='average_passenger_count', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text='Passenger<br>count'), column_align='right', column_width=None), ColInfo(var='average_trip_distance', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text='Trip<br>distance'), column_align='right', column_width=None), ColInfo(var='fare_amount', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text='Fare'), column_align='right', column_width=None), ColInfo(var='extra', type=<ColInfoTypeEnum.default: 1>, column_label='Extra', column_align='right', column_width=None), ColInfo(var='mta_tax', type=<ColInfoTypeEnum.default: 1>, column_label='MTA tax', column_align='right', column_width=None), ColInfo(var='tip_amount', type=<ColInfoTypeEnum.default: 1>, column_label='Tip', column_align='right', column_width=None), ColInfo(var='tolls_amount', 

In [7]:
low_col = Color('lime') - Saturation(0.4)
high_col = Color('yellow') - Saturation(0.2)

In [8]:
gt2 = (
    gt1
    .data_color(
        palette=[low_col.html, high_col.html], 
        columns=[k for k,v in bw.schema.items() 
                 if v.is_numeric() 
                 and k not in ('average_passenger_count', 'average_trip_distance')])
    .data_color(
        palette=["white", "blue"], 
        columns=['average_passenger_count', 'average_trip_distance']
    )
)

In [9]:
gt2

GT(_tbl_data=shape: (5, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ week      ┆ average_p ┆ average_t ┆ fare_amou ┆ … ┆ tip_amoun ┆ tolls_amo ┆ total_amo ┆ passenge │
│ ---       ┆ assenger_ ┆ rip_dista ┆ nt        ┆   ┆ t         ┆ unt       ┆ unt       ┆ r_count_ │
│ datetime[ ┆ count     ┆ nce       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ daily    │
│ μs]       ┆ ---       ┆ ---       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---      │
│           ┆ f64       ┆ f64       ┆           ┆   ┆           ┆           ┆           ┆ list[i64 │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ]        │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2014-12-2 ┆ 1.766969  ┆ 19.393444 ┆ 12.259514 ┆ … ┆ 1.33035   ┆ 0.264227  ┆ 14.909873 ┆ [690407, │
│ 9         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 606716,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 565709]  │
│ 2015-01-0 ┆ 1.675578  ┆ 22.713874 ┆ 11.767176 ┆ … ┆ 1.512062  ┆ 0.239516  ┆ 14.637098 ┆ [600760, │
│ 5         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 632469,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 719033]  │
│ 2015-01-1 ┆ 1.671457  ┆ 2.75792   ┆ 11.890466 ┆ … ┆ 2.841937  ┆ 0.248048  ┆ 16.094246 ┆ [648214, │
│ 2         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 734847,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 743288]  │
│ 2015-01-1 ┆ 1.667416  ┆ 18.76329  ┆ 11.924381 ┆ … ┆ 1.612231  ┆ 0.253735  ┆ 14.886225 ┆ [575507, │
│ 9         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 661009,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 694420]  │
│ 2015-01-2 ┆ 1.665861  ┆ 4.784368  ┆ 11.855754 ┆ … ┆ 1.592241  ┆ 0.214637  ┆ 14.783954 ┆ [375606, │
│ 6         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 232108,  │
│ 00:00:00  ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ …        │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ 898122]  │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x11092a690>, _boxhead=Boxhead([ColInfo(var='week', type=<ColInfoTypeEnum.stub: 2>, column_label='week', column_align='right', column_width=None), ColInfo(var='average_passenger_count', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text='Passenger<br>count'), column_align='right', column_width=None), ColInfo(var='average_trip_distance', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text='Trip<br>distance'), column_align='right', column_width=None), ColInfo(var='fare_amount', type=<ColInfoTypeEnum.default: 1>, column_label=Html(text='Fare'), column_align='right', column_width=None), ColInfo(var='extra', type=<ColInfoTypeEnum.default: 1>, column_label='Extra', column_align='right', column_width=None), ColInfo(var='mta_tax', type=<ColInfoTypeEnum.default: 1>, column_label='MTA tax', column_align='right', column_width=None), ColInfo(var='tip_amount', type=<ColInfoTypeEnum.default: 1>, column_label='Tip', column_align='right', column_width=None), ColInfo(var='tolls_amount', 

In [16]:
bh = (
    df
    .sort(pl.col('tpep_pickup_datetime'), descending=False)
    .group_by_dynamic(
        pl.col('tpep_pickup_datetime').str.to_datetime('%Y-%m-%d %H:%M:%S'), 
        every="1d", 
        group_by=['VendorID'])
    .agg(pl.col('total_amount').mean().alias('total_amount'))
    .group_by('VendorID').agg(pl.col('total_amount').implode())
    .with_columns(pl.col('total_amount').list.get(0))
    .sort('VendorID', descending=False)
)

In [17]:
bh.style.fmt_nanoplot(columns="total_amount", reference_line="mean")

GT(_tbl_data=shape: (2, 2)
┌──────────┬─────────────────────────────────┐
│ VendorID ┆ total_amount                    │
│ ---      ┆ ---                             │
│ i64      ┆ list[f64]                       │
╞══════════╪═════════════════════════════════╡
│ 1        ┆ [15.189741, 14.728701, … 13.91… │
│ 2        ┆ [15.384171, 14.949735, … 14.04… │
└──────────┴─────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x10586c150>, _boxhead=Boxhead([ColInfo(var='VendorID', type=<ColInfoTypeEnum.default: 1>, column_label='VendorID', column_align='right', column_width=None), ColInfo(var='total_amount', type=<ColInfoTypeEnum.default: 1>, column_label='total_amount', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x310677150>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x31067d510>, _formats=[<great_tables._gt_data.FormatInfo object at 0x310674810>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), hea

In [18]:
bh.style.fmt_nanoplot(columns="total_amount", autoscale=True)

GT(_tbl_data=shape: (2, 2)
┌──────────┬─────────────────────────────────┐
│ VendorID ┆ total_amount                    │
│ ---      ┆ ---                             │
│ i64      ┆ list[f64]                       │
╞══════════╪═════════════════════════════════╡
│ 1        ┆ [15.189741, 14.728701, … 13.91… │
│ 2        ┆ [15.384171, 14.949735, … 14.04… │
└──────────┴─────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x31069ec50>, _boxhead=Boxhead([ColInfo(var='VendorID', type=<ColInfoTypeEnum.default: 1>, column_label='VendorID', column_align='right', column_width=None), ColInfo(var='total_amount', type=<ColInfoTypeEnum.default: 1>, column_label='total_amount', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x310798c50>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x310675790>, _formats=[<great_tables._gt_data.FormatInfo object at 0x3027cf610>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), hea

In [19]:
bh.style.fmt_nanoplot(columns="total_amount", autoscale=True, plot_type="bar")

GT(_tbl_data=shape: (2, 2)
┌──────────┬─────────────────────────────────┐
│ VendorID ┆ total_amount                    │
│ ---      ┆ ---                             │
│ i64      ┆ list[f64]                       │
╞══════════╪═════════════════════════════════╡
│ 1        ┆ [15.189741, 14.728701, … 13.91… │
│ 2        ┆ [15.384171, 14.949735, … 14.04… │
└──────────┴─────────────────────────────────┘, _body=<great_tables._gt_data.Body object at 0x3027eb8d0>, _boxhead=Boxhead([ColInfo(var='VendorID', type=<ColInfoTypeEnum.default: 1>, column_label='VendorID', column_align='right', column_width=None), ColInfo(var='total_amount', type=<ColInfoTypeEnum.default: 1>, column_label='total_amount', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x3029053d0>, _spanners=Spanners([]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x107d58190>, _formats=[<great_tables._gt_data.FormatInfo object at 0x3027cf790>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), hea